This is an automatic method of establishing scaling and rotation aspects of affine transformation. However, because the code is looking for random features on the images, it is hard to consistently get the same absolute zero every time

In [29]:
import cv2
import numpy as np

# Load images
img1 = cv2.imread('/content/sample1_(59.0051385,155.99516625,80.0)_20250314_192822.jpg')
img2 = cv2.imread('/content/sample1_(59.0051385,157.00244175,80.0)_20250314_192738.jpg')
img3 = cv2.imread('/content/sample1_(59.999160375,157.00244175,80.0)_20250314_192536.jpg')

# Convert to grayscale for feature detection
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
gray3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

# Initialize ORB detector
orb = cv2.ORB_create()

# Detect features in all images
kp1, des1 = orb.detectAndCompute(gray1, None)
kp2, des2 = orb.detectAndCompute(gray2, None)
kp3, des3 = orb.detectAndCompute(gray3, None)

# Create BFMatcher with cross-check
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match Image1 with Image2 and Image3
matches_12 = bf.match(des1, des2)
matches_13 = bf.match(des1, des3)

# Create dictionaries for quick lookup
match12_dict = {m.queryIdx: m for m in matches_12}
match13_dict = {m.queryIdx: m for m in matches_13}

# Find common features present in all matches
common_features = []
for q_idx in set(match12_dict.keys()) & set(match13_dict.keys()):
    m12 = match12_dict[q_idx]
    m13 = match13_dict[q_idx]
    total_distance = m12.distance + m13.distance
    common_features.append((
        q_idx,        # Image1 keypoint index
        m12.trainIdx,  # Image2 keypoint index
        m13.trainIdx,  # Image3 keypoint index
        total_distance
    ))

# Sort by match quality (lower distance = better)
common_features.sort(key=lambda x: x[3])

# Select top 1 common feature
top_matches = common_features[:1]

# Draw markers on all images
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0)]  # Red, Green, Blue
marker_type = cv2.MARKER_CROSS
marker_size = 500
thickness = 25

for i, (q_idx, t12_idx, t13_idx, _) in enumerate(top_matches):
    # Image 1
    x1, y1 = map(int, kp1[q_idx].pt)
    cv2.drawMarker(img1, (x1, y1), colors[i],
                 marker_type, marker_size, thickness)

    # Image 2
    x2, y2 = map(int, kp2[t12_idx].pt)
    cv2.drawMarker(img2, (x2, y2), colors[i],
                 marker_type, marker_size, thickness)

    # Image 3
    x3, y3 = map(int, kp3[t13_idx].pt)
    cv2.drawMarker(img3, (x3, y3), colors[i],
                 marker_type, marker_size, thickness)

# Save and display results
cv2.imwrite('image1_3matches.jpg', img1)
cv2.imwrite('image2_3matches.jpg', img2)
cv2.imwrite('image3_3matches.jpg', img3)


[array([3610.0557, 1206.3379], dtype=float32), array([[3630.9575, 1884.1564]], dtype=float32), array([2974.0408, 1925.9602], dtype=float32)]


In [32]:
camera_pts = np.float32([kp1[q_idx].pt, kp2[t12_idx].pt, kp3[t13_idx].pt])
print(camera_pts)

stage_pts = np.float32([[0,0], [0,1], [1,1]])
print(camera_pts)

# Compute affine transformation matrix
T = cv2.getAffineTransform(stage_pts, camera_pts)

# Print the transformation matrix
print("Affine Transformation Matrix:")
print(T)

[[3610.0557 1206.3379]
 [3630.9575 1884.1564]
 [2974.0408 1925.9602]]
[[3610.0557 1206.3379]
 [3630.9575 1884.1564]
 [2974.0408 1925.9602]]
Affine Transformation Matrix:
[[-656.91674805   20.90185547 3610.05566406]
 [  41.80383301  677.81848145 1206.33789062]]
